# Imports and utility functions

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg # images
import numpy as np #numpy
import seaborn as sns
import tensorflow.compat.v2 as tf #use tensorflow v2 as a main 
import tensorflow.keras as keras # required for high level applications
from sklearn.model_selection import train_test_split # split for validation sets
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.preprocessing import normalize # normalization of the matrix
import scipy
import pandas as pd
import re



In [ ]:
def cleanTexts(texts):
    cleaned = []
    pattern = "[^a-zA-Z0-9]"
    for text in texts:
        clrd = re.sub(pattern," ",text).lower().strip()
        cleaned.append(clrd)
    return cleaned

# Load data

In [ ]:
from sklearn.utils import shuffle

dataset = pd.read_csv('train_data_imdb.csv')
dataset = shuffle(dataset)
dataset.tail()

,Unnamed: 0,text,label
7719,23629,"I have a high tolerance level for crap, so I w...",0
9361,8113,"In my opinion, the best movie ever. I love whe...",1
13984,3936,This episode of Twilight Zone combines a silen...,1
2840,21728,This movie is nothing but a religious tract pr...,0
11664,11096,"John Van Druten's ""Bell, Book and Candle"" is a...",1


In [ ]:
x_train = list(cleanTexts(dataset['text']))
# print(x[:5])

y_train = list(dataset['label'])
# print(y[:5])

#x_train = x_train[:10000]
#y_train = y_train[:10000]

x_train = x_train[:25000]
y_train = y_train[:25000]

In [ ]:
test_data = pd.read_csv('test_data_imdb.csv')

x_test = list(cleanTexts(test_data['text']))
y_test = list(test_data['label'])

x_test = x_test[:10000]
y_test = y_test[:10000]

# Train model and predict on test dataset

In [ ]:
from tensorflow import string as tf_string
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.compat.v1.keras.layers import CuDNNGRU, CuDNNLSTM
from tensorflow.keras.layers import LSTM, GRU, Bidirectional


## 1-4 Vectorizer parameters

In [ ]:
embedding_dim = 256 # Dimension of embedded representation - 
vocab_size = 30000 # Number of unique tokens in vocabulary
sequence_length = 256 # Output dimension after vectorizing - 


vect_layer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(x_train)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=69, stratify=y_train)

## 1.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True)(x_v)
x = LSTM(128, activation='relu', return_sequences=True)(emb)
x = GRU(128, activation='relu', return_sequences=True)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, 'relu')(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 256)              0         
 ectorization)                                                   
                                                                 
 embedding_3 (Embedding)     (None, 256, 256)          7680000   
                                                                 
 lstm_3 (LSTM)               (None, 256, 128)          197120    
                                                                 
 gru_2 (GRU)                 (None, 256, 128)          99072     
                                                                 
 flatten_3 (Flatten)         (None, 32768)             0         
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 5
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
12/12 [==============================] - 33s 3s/step - loss: 0.1443 - accuracy: 0.9551 - val_loss: 0.4277 - val_accuracy: 0.8400
Epoch 2/2
12/12 [==============================] - 26s 2s/step - loss: 0.0582 - accuracy: 0.9842 - val_loss: 0.5971 - val_accuracy: 0.8420


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 39s 125ms/step - loss: 0.7538 - accuracy: 0.8082


[0.7538464665412903, 0.8082000017166138]

In [ ]:
y_pred=model.predict(x_test)


accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 80.82000000000001% 
f1-score is 0.8125488663017982% 
              precision    recall  f1-score   support

           0       0.82      0.79      0.80      4984
           1       0.80      0.83      0.81      5016

    accuracy                           0.81     10000
   macro avg       0.81      0.81      0.81     10000
weighted avg       0.81      0.81      0.81     10000



## 2.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True)(x_v)
x = Bidirectional(LSTM(128, activation='relu', return_sequences=True))(emb)
x = Bidirectional(GRU(128, activation='relu', return_sequences=True))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, 'relu')(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 256)              0         
 ectorization)                                                   
                                                                 
 embedding_4 (Embedding)     (None, 256, 256)          7680000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 256, 256)         394240    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 256, 256)         296448    
 nal)                                                            
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 5
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/5
12/12 [==============================] - 69s 5s/step - loss: 0.6918 - accuracy: 0.5233 - val_loss: 0.6731 - val_accuracy: 0.6370
Epoch 2/5
12/12 [==============================] - 51s 4s/step - loss: 0.6114 - accuracy: 0.6832 - val_loss: 0.4629 - val_accuracy: 0.8150
Epoch 3/5
12/12 [==============================] - 51s 4s/step - loss: 0.3605 - accuracy: 0.8538 - val_loss: 0.3182 - val_accuracy: 0.8710
Epoch 4/5
12/12 [==============================] - 57s 5s/step - loss: 0.1579 - accuracy: 0.9446 - val_loss: 0.3699 - val_accuracy: 0.8620
Epoch 5/5
12/12 [==============================] - 54s 5s/step - loss: 0.0667 - accuracy: 0.9823 - val_loss: 0.4773 - val_accuracy: 0.8650


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 78s 249ms/step - loss: 0.5919 - accuracy: 0.8382


[0.5919193625450134, 0.8381999731063843]

In [ ]:
y_pred=model.predict(x_test)


accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 83.82% 
f1-score is 0.8423001949317739% 
              precision    recall  f1-score   support

           0       0.85      0.81      0.83      4984
           1       0.82      0.86      0.84      5016

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000



## 3.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True)(x_v)
x = LSTM(128, activation='relu', return_sequences=True)(emb)
x = GRU(128, activation='relu', return_sequences=True)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, 'relu')(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 256)              0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 256, 256)          7680000   
                                                                 
 lstm_1 (LSTM)               (None, 256, 128)          197120    
                                                                 
 gru (GRU)                   (None, 256, 128)          99072     
                                                                 
 flatten_1 (Flatten)         (None, 32768)             0         
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 3
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/3
30/30 [==============================] - 68s 2s/step - loss: 0.5863 - accuracy: 0.6791 - val_loss: 0.3752 - val_accuracy: 0.8392
Epoch 2/3
30/30 [==============================] - 64s 2s/step - loss: 0.2853 - accuracy: 0.8895 - val_loss: 0.3116 - val_accuracy: 0.8704
Epoch 3/3
30/30 [==============================] - 63s 2s/step - loss: 0.1505 - accuracy: 0.9518 - val_loss: 0.3551 - val_accuracy: 0.8696


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 38s 122ms/step - loss: 0.4170 - accuracy: 0.8361


[0.41699403524398804, 0.8360999822616577]

In [ ]:
y_pred=model.predict(x_test)


accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 83.61% 
f1-score is 0.8386016740521911% 
              precision    recall  f1-score   support

           0       0.84      0.82      0.83      4984
           1       0.83      0.85      0.84      5016

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000



## 4.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True)(x_v)
x = Bidirectional(LSTM(128, activation='relu', return_sequences=True))(emb)
x = Bidirectional(GRU(128, activation='relu', return_sequences=True))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, 'relu')(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 256)              0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 256, 256)          7680000   
                                                                 
 bidirectional (Bidirectiona  (None, 256, 256)         394240    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 256, 256)         296448    
 nal)                                                            
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 3
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/3
30/30 [==============================] - 142s 5s/step - loss: 0.6533 - accuracy: 0.5999 - val_loss: 0.4491 - val_accuracy: 0.8140
Epoch 2/3
30/30 [==============================] - 132s 4s/step - loss: 0.3237 - accuracy: 0.8716 - val_loss: 0.2876 - val_accuracy: 0.8832
Epoch 3/3
30/30 [==============================] - 130s 4s/step - loss: 0.1480 - accuracy: 0.9504 - val_loss: 0.3053 - val_accuracy: 0.8896


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 77s 247ms/step - loss: 0.3640 - accuracy: 0.8564


[0.3639925420284271, 0.8564000129699707]

In [ ]:
y_pred=model.predict(x_test)


accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 85.64% 
f1-score is 0.8563137882729638% 
              precision    recall  f1-score   support

           0       0.85      0.86      0.86      4984
           1       0.86      0.85      0.86      5016

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



## 5-6 Vectorizer parameters

In [ ]:
embedding_dim = 512 # Dimension of embedded representation 
vocab_size = 50000 # Number of unique tokens in vocabulary
sequence_length = 512 # Output dimension after vectorizing 


vect_layer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(x_train)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=69, stratify=y_train)

## 5.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True)(x_v)
x = LSTM(256, activation='relu', return_sequences=True)(emb)
x = GRU(128, activation='relu', return_sequences=True)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, 'relu')(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.5)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 512)              0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 512, 512)          25600000  
                                                                 
 lstm_1 (LSTM)               (None, 512, 256)          787456    
                                                                 
 gru_1 (GRU)                 (None, 512, 128)          148224    
                                                                 
 flatten (Flatten)           (None, 65536)             0         
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 3
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/3
30/30 [==============================] - 88s 3s/step - loss: 0.1914 - accuracy: 0.9414 - val_loss: 0.3724 - val_accuracy: 0.8560
Epoch 2/3
30/30 [==============================] - 88s 3s/step - loss: 0.1411 - accuracy: 0.9603 - val_loss: 0.3855 - val_accuracy: 0.8600
Epoch 3/3
30/30 [==============================] - 87s 3s/step - loss: 0.0917 - accuracy: 0.9769 - val_loss: 0.4377 - val_accuracy: 0.8572


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 71s 226ms/step - loss: 0.5325 - accuracy: 0.8394


[0.5324867963790894, 0.8393999934196472]

In [ ]:
y_pred=model.predict(x_test)


accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 83.94% 
f1-score is 0.8389167502507522% 
              precision    recall  f1-score   support

           0       0.83      0.85      0.84      4984
           1       0.84      0.83      0.84      5016

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000



## 6.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True)(x_v)
x = Bidirectional(LSTM(128, activation='relu', return_sequences=True))(emb)
x = Bidirectional(GRU(64, activation='relu', return_sequences=True))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, 'relu')(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.5)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 512)              0         
 torization)                                                     
                                                                 
 embedding_4 (Embedding)     (None, 512, 512)          25600000  
                                                                 
 bidirectional_4 (Bidirectio  (None, 512, 256)         656384    
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 512, 128)         123648    
 nal)                                                            
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 8
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/8
30/30 [==============================] - 175s 6s/step - loss: 0.6929 - accuracy: 0.5149 - val_loss: 0.6905 - val_accuracy: 0.5540
Epoch 2/8
30/30 [==============================] - 171s 6s/step - loss: 0.6751 - accuracy: 0.6164 - val_loss: 0.6384 - val_accuracy: 0.6760
Epoch 3/8
30/30 [==============================] - 169s 6s/step - loss: 0.5573 - accuracy: 0.7522 - val_loss: 0.4344 - val_accuracy: 0.8304
Epoch 4/8
30/30 [==============================] - 169s 6s/step - loss: 0.3680 - accuracy: 0.8622 - val_loss: 0.3091 - val_accuracy: 0.8736
Epoch 5/8
30/30 [==============================] - 171s 6s/step - loss: 0.2531 - accuracy: 0.9112 - val_loss: 0.2756 - val_accuracy: 0.8876
Epoch 6/8
30/30 [==============================] - 168s 6s/step - loss: 0.1895 - accuracy: 0.9386 - val_loss: 0.2602 - val_accuracy: 0.8976
Epoch 7/8
30/30 [==============================] - 171s 6s/step - loss: 0.1379 - accuracy: 0.9588 - val_loss: 0.2721 - val_accuracy: 0.8996
Epoch 8/8
30/30 [===

In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 128s 409ms/step - loss: 0.3863 - accuracy: 0.8692


[0.38628536462783813, 0.8691999912261963]

In [ ]:
y_pred=model.predict(x_test)


accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 86.92% 
f1-score is 0.8738669238187079% 
              precision    recall  f1-score   support

           0       0.90      0.83      0.86      4984
           1       0.85      0.90      0.87      5016

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



# Save model

In [ ]:
model.save('FR_LSTM_sixth')

INFO:tensorflow:Assets written to: FR_LSTM_sixth/assets


In [ ]:
!zip -r /content/FrLSTMsixth.zip /content/FR_LSTM_sixth/

  adding: content/FR_LSTM_sixth/ (stored 0%)
  adding: content/FR_LSTM_sixth/variables/ (stored 0%)
  adding: content/FR_LSTM_sixth/variables/variables.index (deflated 67%)
  adding: content/FR_LSTM_sixth/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/FR_LSTM_sixth/assets/ (stored 0%)
  adding: content/FR_LSTM_sixth/keras_metadata.pb (deflated 90%)
  adding: content/FR_LSTM_sixth/saved_model.pb (deflated 78%)
